<a href="https://colab.research.google.com/github/OswaldoMoper/Mathematical_Data_Analysis/blob/master/Data%20Analysis/Analysis_5_(_Recommendation_systems_)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from scipy import spatial
import operator

In [ ]:
movies_df = pd.read_csv( '/content/movies.csv' )
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
rating_df = pd.read_csv( '/content/ratings.csv' )
rating_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


#Limpieza de Datos

In [ ]:
movies_df['title'] = movies_df.title.str[:-7]
movies_df.head()

,movieId,title,genres
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance
4,5,Father of the Bride Part II,Comedy


In [ ]:
movies_df[ 'genres' ] = ( [ genre.split( '|' ) for genre in movies_df.genres ] )
movies_df.head()

,movieId,title,genres
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji,"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men,"[Comedy, Romance]"
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II,[Comedy]


In [ ]:
gen = []
for genre in movies_df['genres']:
    gen.extend(genre)
gen = list(set(gen))
gen.sort()
gen_vec = np.zeros(len(gen), dtype = int)
genre = []
for genres in movies_df['genres']:
    gv = np.zeros(len(gen), dtype = int)
    for i in range(len(genres)):
        for j in range(len(gen)):
            if genres[i] == gen[j]:
                gv[j] += 1
                gen_vec[j] += 1
                break
    genre.append(gv)
movies_df['genres_v'] = pd.Series(genre)
movies_df

,movieId,title,genres,genres_v
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]","[0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1,2,Jumanji,"[Adventure, Children, Fantasy]","[0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
2,3,Grumpier Old Men,"[Comedy, Romance]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]","[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,5,Father of the Bride Part II,[Comedy],"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,"[Action, Animation, Comedy, Fantasy]","[0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
9738,193583,No Game No Life: Zero,"[Animation, Comedy, Fantasy]","[0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
9739,193585,Flint,[Drama],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
9740,193587,Bungo Stray Dogs: Dead Apple,"[Action, Animation]","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
df = rating_df.merge( movies_df, on = 'movieId' )
df = df[ [ 'userId', 'title', 'movieId', 'rating', 'genres', 'genres_v' ] ]
df = df.sort_values( [ 'userId', 'rating', 'title' ], ascending = [ True, False, True ] ).reset_index( drop = True )
df[ 'userId' ] = df.userId.astype( str )
df.head()

,userId,title,movieId,rating,genres,genres_v
0,1,"Adventures of Robin Hood, The",940,5.0,"[Action, Adventure, Romance]","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,Alice in Wonderland,1032,5.0,"[Adventure, Animation, Children, Fantasy, Musi...","[0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, ..."
2,1,All Quiet on the Western Front,1927,5.0,"[Action, Drama, War]","[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,1,American Beauty,2858,5.0,"[Drama, Romance]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,1,American History X,2329,5.0,"[Crime, Drama]","[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
df.shape

(100836, 6)

#Analisis Exploratorio

In [ ]:
movies = pd.DataFrame( df.title.value_counts() ).reset_index()
movies.columns = [ 'Title', 'Count' ]
movies

,Title,Count
0,Forrest Gump,329
1,"Shawshank Redemption, The",317
2,Pulp Fiction,307
3,"Silence of the Lambs, The",279
4,"Matrix, The",278
...,...,...
9440,"Sunset Limited, The",1
9441,Tabu,1
9442,The End of the Tour,1
9443,The Forgotten Space,1


In [ ]:
top_n = 15
figt = px.bar( movies.head( top_n ), x = 'Title', y = 'Count', color = 'Title', text = 'Count'
             , title = 'Top ' + str(top_n) + ' - Películas Más Vistas', template = 'plotly_dark' )
figt.update_layout( showlegend = False )
figt.show()

In [ ]:
users =  pd.DataFrame( df.userId.value_counts() ).reset_index()
users.columns = [ 'UserId', 'Freq' ]
users.head()

,UserId,Freq
0,414,2698
1,599,2478
2,474,2108
3,448,1864
4,274,1346


In [ ]:
top_n = 10
figtu = px.bar( users.head( top_n ), x = 'UserId', y = 'Freq', color = 'UserId', text = 'Freq'
              , title = 'Top ' + str( top_n ) + ' - Usuarios más Activos', template = 'plotly_dark' )
figtu.update_layout( showlegend = False )
figtu.show()

In [ ]:
figh = px.histogram( users, x = 'Freq', marginal = 'box', template = 'plotly_dark', title = 'Histograma - Actividad de Usuarios' )
figh.update_layout( bargap = 0.12 )
figh.show()

In [ ]:
users.Freq.describe()

count     610.000000
mean      165.304918
std       269.480584
min        20.000000
25%        35.000000
50%        70.500000
75%       168.000000
max      2698.000000
Name: Freq, dtype: float64

In [ ]:
movies_ratings = df[ [ 'title', 'rating' ] ].groupby( [ 'title' ] ).agg( { 'title':'size', 'rating':'mean' } )
movies_ratings.columns = [ 'Freq', 'Rating' ]
movies_ratings = movies_ratings.reset_index()
movies_ratings = movies_ratings.sort_values( [ 'Freq', 'Rating', 'title' ], ascending = [ False, False, True ] ).reset_index( drop = True )
movies_ratings.head( 10 )

,title,Freq,Rating
0,Forrest Gump,329,4.164134
1,"Shawshank Redemption, The",317,4.429022
2,Pulp Fiction,307,4.197068
3,"Silence of the Lambs, The",279,4.161290
4,"Matrix, The",278,4.192446
5,Star Wars: Episode IV - A New Hope,251,4.231076
6,Jurassic Park,238,3.750000
7,Braveheart,237,4.031646
8,Terminator 2: Judgment Day,224,3.970982
9,Schindler's List,220,4.225000


In [ ]:
fighr = px.histogram( movies_ratings, x = 'Rating', marginal = 'box', nbins = 10, template = 'plotly_dark', title = 'Histograma - Calificación por Películas'  )
fighr.update_layout( bargap = 0.12 )
fighr.show()

In [ ]:
figb = px.bar( x = gen, y = gen_vec, color = gen, text = gen_vec, title = 'Género de Películas', template = 'plotly_dark' )
figb.update_layout( showlegend = False )
figb.show()

#Similar Users

In [ ]:
ratings = df[ [ 'userId', 'movieId', 'title', 'rating' ] ]
ratings = ratings.merge( users, left_on = 'userId', right_on = 'UserId' ).drop( [ 'UserId' ], axis = 1 )
ratings

,userId,movieId,title,rating,Freq
0,1,940,"Adventures of Robin Hood, The",5.0,232
1,1,1032,Alice in Wonderland,5.0,232
2,1,1927,All Quiet on the Western Front,5.0,232
3,1,2858,American Beauty,5.0,232
4,1,2329,American History X,5.0,232
...,...,...,...,...,...
100831,610,8840,Who's That Knocking at My Door?,1.0,1302
100832,610,68319,X-Men Origins: Wolverine,1.0,1302
100833,610,145951,Bloodsport: The Dark Kumite,0.5,1302
100834,610,61818,"Crow, The: Wicked Prayer",0.5,1302


In [ ]:
#df[ df.title.isin( movies.head( top_n ).title.unique() ) ] obtener la tabla anterior pero solo del topn

In [ ]:
userRatings = ratings.pivot_table( index = [ 'userId' ], columns = [ 'title' ], values = 'rating' ).reset_index( drop = True )
userRatings

title,,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,...,Zulu,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.5,3.0,NaN,NaN,NaN
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# userRatings.melt( id_vars = ['userId'] )

In [ ]:
corrMatrix = userRatings.corr( method = 'pearson', min_periods = 75 )
corrMatrix

title,,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,...,Zulu,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
title,,,,,,,,,,,,,,,,,,,,,
,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Hellboy': The Seeds of Creation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xXx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xXx: State of the Union,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def Recomendacion_Usuario(user_id):
  myRatings = userRatings.loc[ user_id ].dropna()
  myRatings.sort_values(inplace = True, ascending = False)
  print('-> Usuario {} estas son algunas peliculas que más te han gustado:'.format(user_id))
  for i in range(10):
    print('{}. {}'.format(i+1, myRatings.index[i]))
  print('\n-> Estoy buscando las mejores recomendaciones para ti, espera por favor...')
  simCandidates = pd.Series(dtype = 'float64')
  for i in range(0, len(myRatings)):
    sims = corrMatrix[myRatings.index[i]].dropna()
    sims = sims.map(lambda x: x * myRatings[i])
    simCandidates = pd.concat([simCandidates, sims])
  simCandidates.sort_values(inplace = True, ascending = False)
  simCandidates = simCandidates.groupby(simCandidates.index).sum()
  simCandidates.sort_values(inplace = True, ascending = False)
  filteredSims = simCandidates.drop(myRatings.index, errors = 'ignore')
  print('->Gracias por esperar, estas son unas películas que te pueden gustar: ')
  if(len(filteredSims < 10)):
    for i in range((len(filteredSims))):
      print('{}. {}'.format(i+1, filteredSims.index[i]))
  else:
    for i in range(10):
      print('{}.{}'.format(i+1, filteredSims.index[i]))

In [ ]:
user_id = input( '¡Bienvenido!\nPor favor ingresa tu número de usuario: ' )
Recomendacion_Usuario( int( user_id ) )

¡Bienvenido!
Por favor ingresa tu número de usuario: 1
-> Usuario 1 estas son algunas peliculas que más te han gustado:
1. First Daughter
2. Skyfall
3. Spectre
4. Dark Knight Rises, The
5. King's Speech, The
6. Despicable Me
7. Batman Begins
8. Intouchables
9. The Intern
10. Notebook, The

-> Estoy buscando las mejores recomendaciones para ti, espera por favor...
->Gracias por esperar, estas son unas películas que te pueden gustar: 
1. Pirates of the Caribbean: The Curse of the Black Pearl
2. Star Wars: Episode V - The Empire Strikes Back
3. Memento
4. Silence of the Lambs, The
5. Star Wars: Episode IV - A New Hope
6. Toy Story
7. Star Wars: Episode VI - Return of the Jedi
8. Shawshank Redemption, The
9. Monsters, Inc.
10. Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark)
11. Braveheart
12. Lion King, The
13. Kill Bill: Vol. 1
14. Jurassic Park
15. Inception
16. Indiana Jones and the Last Crusade
17. Saving Private Ryan
18. Terminator 2: Judgment Day
19. Godfather

#Similar Movies

In [ ]:
movieProp =  df.groupby( 'movieId' ).agg( { 'rating':[ np.size, np.mean ] } )
movieFreq = pd.DataFrame( movieProp[ 'rating' ][ 'size' ] )
movieFreqNorm = movieFreq.apply( lambda x: ( x -np.min( x ) ) / ( np.max( x ) - np.min( x ) ) )
movieRatings = movies_df.merge( movieFreqNorm.reset_index(), on = 'movieId' )
movieProp = movieProp.reset_index()
movieProp.columns = [ 'movieId', 'Freq', 'Rating' ]
movieRatings = movieRatings.merge( movieProp, on = 'movieId' )
movieRatings

,movieId,title,genres,genres_v,size,Freq,Rating
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]","[0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0.652439,215,3.920930
1,2,Jumanji,"[Adventure, Children, Fantasy]","[0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0.332317,110,3.431818
2,3,Grumpier Old Men,"[Comedy, Romance]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.155488,52,3.259615
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]","[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.018293,7,2.357143
4,5,Father of the Bride Part II,[Comedy],"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.146341,49,3.071429
...,...,...,...,...,...,...,...
9719,193581,Black Butler: Book of the Atlantic,"[Action, Animation, Comedy, Fantasy]","[0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0.000000,1,4.000000
9720,193583,No Game No Life: Zero,"[Animation, Comedy, Fantasy]","[0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0.000000,1,3.500000
9721,193585,Flint,[Drama],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.000000,1,3.500000
9722,193587,Bungo Stray Dogs: Dead Apple,"[Action, Animation]","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.000000,1,3.500000


In [ ]:
movieDict = {}
for i in range( len( movieRatings[ 'movieId' ] ) ):
  movieDict[ movieRatings[ 'movieId' ][ i ] ] = ( movieRatings[ 'title' ][ i ], movieRatings[ 'genres_v' ][ i ]
                                                , movieRatings[ 'size' ][ i ], movieRatings[ 'Rating' ][ i ] )
movieDict

{1: ('Toy Story',
  array([0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  0.6524390243902439,
  3.9209302325581397),
 2: ('Jumanji',
  array([0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  0.3323170731707317,
  3.4318181818181817),
 3: ('Grumpier Old Men',
  array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]),
  0.15548780487804878,
  3.2596153846153846),
 4: ('Waiting to Exhale',
  array([0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]),
  0.018292682926829267,
  2.357142857142857),
 5: ('Father of the Bride Part II',
  array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  0.14634146341463414,
  3.0714285714285716),
 6: ('Heat',
  array([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]),
  0.3079268292682927,
  3.946078431372549),
 7: ('Sabrina',
  array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]),
  0.16158536585365854,
  3.185185185185185),
 8: ('Tom and Huck',
  array([0, 0, 

In [ ]:
def Distancia( a, b ):
    genresA = a[ 1 ]
    genresB = b[ 1 ]
    genreDistance = spatial.distance.cosine( genresA, genresB )
    popularityA = a[ 2 ]
    popularityB = b[ 2 ]
    popularityDistance = abs( popularityA - popularityB )
    return genreDistance + popularityDistance

In [ ]:
def Peliculas_Similares( movieID, K ):
    distances = []
    for movie in movieDict:
        if ( movie != movieID ):
            dist = Distancia( movieDict[ movieID ], movieDict[ movie ] )
            distances.append( ( movie, dist ) )
    distances.sort( key = operator.itemgetter( 1 ) )
    neighbors = []
    for x in range( K ):
        neighbors.append( distances[ x ][ 0 ] )
    return neighbors

In [ ]:
def Recomendacion_Pelicula( movie_id ):
    print( '\n-> ¡{} es una excelente película!'.format( movieDict[ movie_id ][ 0 ] ) )
    print( '\n-> Estoy buscando peliculas similares a {}, no tardaré...'.format( movieDict[ movie_id ][ 0 ] ) )
    K = 25
    avgRating = 0
    recommendations = Peliculas_Similares( movie_id, K )
    recomendaciones = []
    for recommendation in recommendations:
        avgRating += movieDict[ recommendation ][ 3 ]
        recomendaciones.append( [ movieDict[ recommendation ][ 0 ], movieDict[ recommendation ][ 3 ] ] )
    rec = pd.DataFrame( recomendaciones, columns = [ 'Title', 'Score' ] )
    rec = rec.sort_values( by = [ 'Score' ] ).reset_index( drop = True )
    print( '\n-> Gracias por esperar, estas son unas películas que creo te pueden gustar:' )
    for i in range( 10 ):
        print( " {}. {}".format( i+1, rec[ 'Title' ][ i ] ) )

In [ ]:
movie_id = input( '¡Bienvenido!\nPor favor ingresa el ID de una película: ' )
Recomendacion_Pelicula( int( movie_id ) )

¡Bienvenido!
Por favor ingresa el ID de una película: 13

-> ¡Balto es una excelente película!

-> Estoy buscando peliculas similares a Balto, no tardaré...

-> Gracias por esperar, estas son unas películas que creo te pueden gustar:
 1. Pocahontas II: Journey to a New World 
 2. Gulliver's Travels
 3. Brother Bear
 4. Digimon: The Movie
 5. Dinosaur
 6. Rio
 7. Jimmy Neutron: Boy Genius
 8. Land Before Time, The
 9. Transformers: The Movie
 10. Song of the South
